In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import sklearn
from sklearn.linear_model import LogisticRegressionCV,LinearRegression
from sklearn.linear_model.coordinate_descent import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
## 设置字符集,防止中文乱码
mpl.rcParams['font.sans-serif']=['simHei']
mpl.rcParams['axes.unicode_minus']=False

## 异常拦截
warnings.filterwarnings(action='ignore',category=ConvergenceWarning)

## 数据读取并处理异常数据

In [12]:
## 数据读取并处理异常数据
path='datas/breast-cancer-wisconsin.data'
names=['id','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape',
        'Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei',
        'Bland Chromation','Normal Nucleoli','Mitoses','Class']
df=pd.read_csv(path,header=None,names=names)
df.head()

,id,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromation,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [13]:
datas=df.replace('?',np.nan).dropna(how='any')
datas.head()

,id,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromation,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


## 数据提取

In [50]:
X=datas[names[1:10]]
Y=datas[names[10]]

## 数据集划分

In [51]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

## 数据归一化

注意:最好只针对连续型数据

In [52]:
ss=StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

## 模型构建及训练

+ solver: 参数优化方式
+ 当penalty为l1的时候，参数只能是：liblinear(坐标轴下降法)；
+ nlbfgs和cg都是关于目标函数的二阶泰勒展开
+ 当penalty为l2的时候，参数可以是：lbfgs(拟牛顿法)、newton-cg(牛顿法变种)，seg(minibatch)
+ 维度<10000时，lbfgs   维度>10000时， cg法比较好，显卡计算的时候，lbfgs和cg都比seg快
+ multi_class: 分类方式参数；参数可选: ovr(默认)、multinomial；这两种方式在二元分类问题中，效果是一样的；在多元分类问题中，效果不一样
+ ovr: one-vs-rest， 对于多元分类的问题，先将其看做二元分类，分类完成后，再迭代对其中一类继续进行二元分类
+ multinomial: many-vs-many（MVM）,对于多元分类问题，如果模型有T类，我们每次在所有的T类样本里面选择两类样本出来，
+ 不妨记为T1类和T2类，把所有的输出为T1和T2的样本放在一起，把T1作为正例，T2作为负例，
+ 进行二元逻辑回归，得到模型参数。我们一共需要T(T-1)/2次分类

In [53]:
lr = LogisticRegressionCV(multi_class='ovr',fit_intercept=True, Cs=np.logspace(-2, 2, 20), cv=2, penalty='l1', solver='liblinear', tol=0.01)
re=lr.fit(X_train, Y_train)

In [55]:
## 模型效果获取
r=re.score(X_train,Y_train)
print('R值(准确率):',r)
print ("稀疏化特征比率：%.2f%%" % (np.mean(lr.coef_.ravel() == 0) * 100))

R值(准确率): 0.97619047619
稀疏化特征比率：0.00%
